In [40]:
# Import dependencies
import pandas as pd
import pprint
from inspect import getmembers
import csv

In [16]:
from sportsipy.nfl.schedule import Schedule

newengland_schedule = Schedule('NWE', year = '2021')
for game in newengland_schedule:
    print(game.boxscore_index)  # Prints the boxscore URI for each game
    boxscore = game.boxscore  # Returns an instance of the Boxscore class for this specific game

202109120nwe
202109190nyj
202109260nwe
202110030nwe
202110100htx
202110170nwe
202110240nwe
202110310sdg
202111070car
202111140nwe
202111180atl
202111280nwe
202112060buf
202112190clt
202112260nwe
202201020nwe
202201090mia


In [4]:
from sportsipy.nfl.boxscore import Boxscore

game_data = Boxscore('202109120nwe')
#print(game_data.home_points)  # Prints 33
#print(game_data.away_points)  # Prints 41
#df = game_data.dataframe  # Returns a Pandas DataFrame of game metrics
#df
# pprint.pprint(dir(game_data.home_players[0]))
game_data

Boxscore for Miami Dolphins at New England Patriots (Sunday Sep 12, 2021)

In [6]:
def home_away_split(game_data):
    #Home
    home_df = game_data.home_players[0].dataframe
    for player in game_data.home_players[1:]:
        home_df = pd.concat([home_df, player.dataframe], axis = 0)
    home_df['name'] = [x.name for x in game_data.home_players]
    home_df.set_index('name', inplace = True)
    away_df = game_data.away_players[0].dataframe
    
    #Away
    for player in game_data.away_players[1:]:
        away_df = pd.concat([away_df, player.dataframe], axis = 0)
    away_df['name'] = [x.name for x in game_data.away_players]
    away_df.set_index('name', inplace = True)

    return home_df, away_df

In [46]:
home_df.to_csv('home_df.csv')

In [20]:
away_df = game_data.away_players[0].dataframe
for player in game_data.away_players[1:]:
    away_df = pd.concat([away_df, player.dataframe], axis = 0)
away_df['name'] = [x.name for x in game_data.away_players]
away_df.set_index('name', inplace = True)
away_df.head(10)

,completed_passes,attempted_passes,passing_yards,passing_touchdowns,interceptions_thrown,times_sacked,yards_lost_from_sacks,longest_pass,quarterback_rating,rush_attempts,...,punt_return_touchdown,longest_punt_return,extra_points_made,extra_points_attempted,field_goals_made,field_goals_attempted,punts,total_punt_yards,yards_per_punt,longest_punt
name,,,,,,,,,,,,,,,,,,,,,
Tua Tagovailoa,16,27,202,1,1,2,17,36,79.6,4,...,None,None,None,None,None,None,None,None,None,None
Myles Gaskin,0,0,0,0,0,0,0,0,NaN,9,...,None,None,None,None,None,None,None,None,None,None
Malcolm Brown,0,0,0,0,0,0,0,0,NaN,5,...,None,None,None,None,None,None,None,None,None,None
Salvon Ahmed,0,0,0,0,0,0,0,0,NaN,3,...,None,None,None,None,None,None,None,None,None,None
Jacoby Brissett,0,0,0,0,0,0,0,0,NaN,2,...,None,None,None,None,None,None,None,None,None,None
DeVante Parker,0,0,0,0,0,0,0,0,NaN,0,...,None,None,None,None,None,None,None,None,None,None
Jaylen Waddle,0,0,0,0,0,0,0,0,NaN,0,...,None,None,None,None,None,None,None,None,None,None
Durham Smythe,0,0,0,0,0,0,0,0,NaN,0,...,None,None,None,None,None,None,None,None,None,None
Albert Wilson,0,0,0,0,0,0,0,0,NaN,0,...,None,None,None,None,None,None,None,None,None,None


In [41]:
from sportsipy.nfl.roster import Player
from sportsipy.nfl.teams import Teams
from sportsipy.nfl.roster import Roster

# Function to get player info from Player class object.
def get_player_df(player):
    # helper function to get player age during each season.
#     def get_age(year, bd):
#         if year[0] == "Career":
#             return None
#         else:
#             year_dt = datetime(int(year[0][0:4]) + 1, 1, 1)
#             age_years = relativedelta(year_dt, bd).years + relativedelta(year_dt, bd).months/12
#             return age_years
        
    # helper function to get year for each row and denote
    # rows that contain career totals.
    def get_week(x):
        if x[0] == "Week":
            return "Week"
#         elif ix[0] == "1999-00":
#             return "2000"
        else:
            return x[0][0:2] + x[0][-2:]
    # get player df and add some extra info
    player_df = player.dataframe
#     player_df['birth_date'] = player.birth_date
    player_df['player_id'] = player.player_id
    player_df['name'] = player.name
#     player_df['year'] = [get_year(ix) for ix in player_df.index]
    player_df['id'] = [player_id + ' ' + year for player_id, year in zip(player_df['player_id'], player_df['year'])]
#     player_df['age'] = [get_age(year, bd) for year, bd in zip(player_df.index, player_df['birth_date'])]
    player_df.set_index('id', drop = True, inplace = True)
    return player_df

# initialize a list of players that we have pulled data for
players_collected = []
week_df_init = 0
# career_df_init = 0
week_df = 0
# career_df = 0

# iterate through years.
for year in range(2020, 2015, -1):
    print('\n' + str(year))
    
    # iterate through all teams in that year.
    for team in Teams(year = str(year)).dataframes.index:
        print('\n' + team + '\n')
        # iterate through every player on a team roster.
        for player_id in Roster(team, year = year, slim = False).players.keys():
            # only pull player info if that player hasn't been pulled already.
            if player_id not in players_collected:
                player = Player(player_id)
                player_info = get_player_df(player)
                player_seasons = player_info[player_info['year'] != "Career"]
                player_career = player_info[player_info['year'] == "Career"]
                # create season_df if not initialized
                if not season_df_init:
                    season_df = player_seasons
                    season_df_init = 1
            # else concatenate to season_df
            else:
                season_df = pd.concat([season_df, player_seasons], axis = 0)
            if not career_df_init:
                career_df = player_career
                career_df_init = 1
            # else concatenate to career_df
            else:
                career_df = pd.concat([career_df, player_career], axis = 0)
            # add player to players_collected
            players_collected.append(player_id)
            print(player.name)

season_df.to_csv('nfl_player_stats_by_season.csv')
career_df.to_csv('nfl_player_stats_by_career.csv')



2020

GNB


BUF


TAM


OTI


NOR


KAN


RAV


SEA


CLT


RAI


MIN


PIT


CRD


CLE


MIA


ATL


DAL


HTX


SDG


DET


SFO


RAM


CHI


CAR


WAS


PHI


NWE


DEN


CIN


JAX


NYG


NYJ


2019

RAV


SFO


TAM


NOR


KAN


DAL


NWE


MIN


SEA


OTI


RAM


PHI


ATL


HTX


GNB


CRD


CLT


DET


NYG


CAR


SDG


CLE


BUF


RAI


MIA


JAX


PIT


DEN


CHI


CIN


NYJ


WAS


2018

KAN


RAM


NOR


NWE


CLT


SEA


SDG


PIT


CHI


ATL


HTX


TAM


RAV


GNB


CAR


NYG


CIN


PHI


MIN


CLE


SFO


DAL


NYJ


DEN


DET


MIA


OTI


RAI


WAS


BUF


JAX


CRD


2017

RAM


NWE


PHI


NOR


JAX


KAN


DET


PIT


RAV


MIN


SEA


CAR


SDG


DAL


ATL


WAS


HTX


TAM


OTI


SFO


GNB


BUF


RAI


NYJ


CRD


CIN


DEN


MIA


CHI


CLT


NYG


CLE


2016

ATL


NOR


NWE


GNB


DAL


CRD


RAI


CLT


SDG


BUF


PIT


WAS


KAN


OTI


CAR


PHI


MIA


SEA


TAM


DET


RAV


DEN


MIN


CIN


JAX


NYG


SFO


HTX


CHI


NYJ


CLE


RAM


2015

CAR

AttributeError: 'int' object has no attribute 'to_csv'

In [47]:
season_df.head()

AttributeError: 'int' object has no attribute 'head'

In [50]:
from sportsipy.nfl.boxscore import Boxscore

game_data = Boxscore('201802040nwe')
print(game_data.home_points)  # Prints 33
print(game_data.away_points)  # Prints 41
df = game_data.dataframe  # Returns a Pandas DataFrame of game metrics


33
41


In [51]:
from sportsipy.nfl.schedule import Schedule

dallas_schedule = Schedule('DAL')
for game in dallas_schedule:
    print(game.boxscore_index)  # Prints the boxscore URI for each game
    boxscore = game.boxscore  # Returns an instance of the Boxscore class for this specific game


202109090tam
202109190sdg
202109270dal
202110030dal
202110100dal
202110170nwe
202110310min
202111070dal
202111140dal
202111210kan
202111250dal
202112020nor
202112120was
202112190nyg
202112260dal
202201020dal
202201090phi


In [52]:
# from sportsipy.nfl.boxscore import Boxscore

game_sch = Schedule('202109090tam')
#print(game_data.home_points)  # Prints 33
#print(game_data.away_points)  # Prints 41
#df = game_data.dataframe  # Returns a Pandas DataFrame of game metrics
#df
# pprint.pprint(dir(game_data.home_players[0]))
game_sch

HTTPError: HTTP Error 404: Not Found

In [ ]:
from sportsipy.nfl.roster import Player

dallas_player = Schedule('DAL')
for game in dallas_schedule:
    print(game.boxscore_index)  # Prints the boxscore URI for each game
    boxscore = game.boxscore  # Returns an instance of the Boxscore class for this specific game

In [54]:
from sportsipy.nhl.roster import Roster

detroit = Roster('DET')
for player in detroit.players:
    print(player.name, player.games_played)

Tyler Bertuzzi 211
Danny DeKeyser 491
Adam Erne 114
Robby Fabbri 164
Sam Gagner 542
Thomas Greiss None
Filip Hronek 170
Dylan Larkin 435
Nick Leddy 518
Gustav Lindstrom 32
Vladislav Namestnikov 263
Alex Nedeljkovic None
Jordan Oesterle 25
Michael Rasmussen 105
Lucas Raymond 3
Carter Rowney 152
Moritz Seider 3
Givani Smith 40
Marc Staal 892
Troy Stecher 286
Mitchell Stephens 45
Pius Suter 55
Filip Zadina 89


In [57]:
from sportsipy.nfl.roster import Roster
player_df = player.dataframe
player_df

,adjusted_assists,adjusted_goals,adjusted_goals_against_average,adjusted_goals_created,adjusted_points,age,assists,average_time_on_ice,blocks_at_even_strength,corsi_against,...,takeaways,team_abbreviation,ties_plus_overtime_loss,time_on_ice,time_on_ice_even_strength,total_goals_against_on_ice,total_goals_for_on_ice,total_shots,weight,wins
2018-19,2,1,None,1,3,19.0,2,15:23,3,None,...,5,DET,None,138,None,7,5,38,189,None
2019-20,8,9,None,7,17,20.0,7,15:10,10,None,...,12,DET,None,425,None,23,19,107,189,None
2020-21,19,9,None,10,28,21.0,13,16:48,22,None,...,17,DET,None,823,None,27,33,169,189,None
2021-22,27,28,None,23,55,22.0,1,14:50,2,None,...,1,DET,None,45,None,2,2,15,189,None
Career,56,47,None,41,103,NaN,23,16:05,37,None,...,35,None,None,1431,None,59,59,329,189,None
